In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Load Data

In [2]:
data_path = 'C:/Users/ticom/VLabs/Data'
df = pd.read_csv(data_path+'/sales_20_21_train.csv', header=0, parse_dates=['DT_VENDA'])
# df = pd.read_csv('https://raw.githubusercontent.com/marcos-mansur/vlabs-challenge/main/Data/sales_20_21_train.csv', header=0, parse_dates=['DT_VENDA'])
df_sub = pd.read_csv(data_path+'/sample_submission.csv',header=0)

In [23]:
df.head()

,ID_VENDA,DT_VENDA,LOJA,QTD_SKU,VALOR,ID_CLIENTE,CANAL
0,1176550,2020-01-02,37,1,9.50,337763,FIS
1,11683461,2020-01-02,15,1,6.64,195687,FIS
2,3326484,2020-01-02,29,5,446.87,56059,FIS
3,7317475,2020-01-02,18,1,6.13,297533,FIS
4,4366938,2020-01-02,23,1,13.17,181284,FIS


In [26]:
df[df['ID_CLIENTE']==137298]

,ID_VENDA,DT_VENDA,LOJA,QTD_SKU,VALOR,ID_CLIENTE,CANAL
51392,3057916,2020-01-17,28,1,0.49,137298,FIS
1235311,756345,2020-11-27,45,3,12.94,137298,FIS
1287720,975766,2020-12-09,45,18,237.10,137298,FIS
1404365,1154109,2021-01-05,45,3,10.76,137298,FIS


In [25]:
df[df['ID_CLIENTE']==136746]

,ID_VENDA,DT_VENDA,LOJA,QTD_SKU,VALOR,ID_CLIENTE,CANAL
1034087,6168546,2020-10-20,0,2,0.05,136746,ECM


# Preprocessing

In [19]:
def treated_data(df_pp):
    max_date = df_pp['DT_VENDA'].max()
    
    #agrupar por cliente somando o valor
    cust_revenue = df_pp.groupby(['ID_CLIENTE'])['VALOR'].sum().copy()
    #ordenar por data
    cust_revenue.sort_values(ascending=False, inplace=True)
    # transformar em df
    cust_rev = pd.DataFrame(cust_revenue)
    # feature com número de compras feitas no período de teste
    cust_rev['n_compras'] = df_pp.groupby(['ID_CLIENTE'])['ID_VENDA'].count()
    # std entre os valores das compras de cada cliente
    cust_rev['std entre compras'] = df_pp.groupby('ID_CLIENTE')['VALOR'].std()
    # preenche NAN com 0
    #cust_rev['std entre compras'] = cust_rev['std entre compras'].fillna(0)
    # meadia entre os valores das compras de cada cliente
    cust_rev['valor medio'] = df_pp.groupby('ID_CLIENTE')['VALOR'].mean()
    
    # diferença entre a data da primeira e ultima compra
    cust_rev['periodo_compras'] = df_pp.groupby(['ID_CLIENTE'])['DT_VENDA'].max() - df_pp.groupby(['ID_CLIENTE'])['DT_VENDA'].min()
    cust_rev['periodo_compras'] = cust_rev['periodo_compras'].apply(lambda x: x.days)
    # data da ultima compra
    cust_rev['ultima compra'] = max_date - df_pp.groupby(['ID_CLIENTE'])['DT_VENDA'].max()
    cust_rev['ultima compra'] = cust_rev['ultima compra'].apply(lambda x: x.days)
    # soma do VALOR dos ultimos 3 meses de cada cliente
    cust_rev['valor_3m'] = df_pp[
            df_pp['DT_VENDA']>(df_pp['DT_VENDA'].max() -pd.to_timedelta(90, unit='d'))
            ].groupby(['ID_CLIENTE'])['VALOR'].sum()
    
    return cust_rev.fillna(0)

In [16]:
data = df[['ID_CLIENTE','VALOR']].groupby(['ID_CLIENTE'])['VALOR'].sum()

In [17]:
data.head()

ID_CLIENTE
0     636.29
2     662.26
3     215.06
4    1271.87
5      47.19
Name: VALOR, dtype: float64